In [1]:
import cvxpy as cp
import numpy as np
import dccp
import numpy.typing as npt
from enum import Enum
from multireward_ope.tabular.mdp import MDP
from multireward_ope.tabular.characteristic_time import CharacteristicTimeSolver
from multireward_ope.tabular.reward_set import RewardSetCircle, \
    RewardSetRewardFree, RewardSetPolytope, RewardSetFinite
from typing import NamedTuple
from multireward_ope.tabular.envs.riverswim import RiverSwim, RiverSwimParameters
from multireward_ope.tabular.policy import Policy, PolicyFactory
import matplotlib.pyplot as plt
from scipy.special import xlogy, rel_entr

In [4]:
values = []
num_states = range(2,5)
for s in num_states:
    env = RiverSwimParameters(num_states=s).build()

    policy = np.zeros(env.dim_state, dtype=np.ulong)

    policy_stochastic = np.zeros((env.dim_state, env.dim_action))
    policy_stochastic[np.arange(env.dim_state), policy] = 1

    rewards_set = np.zeros((1, env.dim_state))
    rewards_set[0,0]=1
    rewards = RewardSetFinite(env.dim_state, env.dim_action,
                            RewardSetFinite.RewardSetFiniteConfig(rewards_set))
    solver = CharacteristicTimeSolver(env.dim_state, env.dim_action, solver=cp.CLARABEL)

    solver.build_problem(rewards)
    res = solver.solve(0.95, env, policy)
    w_pol = res.w / res.w.sum(-1, keepdims=True)
    err = np.linalg.norm( (w_pol - policy_stochastic), ord=1, axis=-1).mean(0)
    values.append(err)

w_pol

array([[0.5       , 0.5       ],
       [0.70335372, 0.29664628],
       [0.47437582, 0.52562418],
       [1.        , 0.        ]])